In [1]:
"""тренируем автоэнкодер который работает на внутреннем представлениии кодера для зуба ( 256* 32), 
   метки выдает тот же кодер из T2.
"""
import torch

from points_dataset import EmbedderDataset
from alignment_ae_dataset import AlignerDataset
from torch_geometric.data import Data
from torch.utils.data import DataLoader

from models import *
from models_align import *
from point_cloud_dataset import PointCloudDataset
from torch.utils.tensorboard import SummaryWriter
torch.__version__, "cuda available" if torch.cuda.is_available() else "cpu only"

('2.4.0+cu124', 'cuda available')

In [2]:
device = "cpu" if not torch.cuda.is_available() else "cuda"

model_dir = "models_align"
align_ds_path = r"datasets_align\dataset_128.pth"
align_ds = torch.load(align_ds_path)
# align_ds_data = align_ds.data # нет у конкатенрованного ds такого поля 
print(f"Using {device} device, ds length - {len(align_ds)}")

EPOCHS = 560
BATCH_SIZE = 64
SPLIT_FACTOR = .8
# TRAIN_MODE = (False, True)[1]
# REMOVE_OLD_MODELS = True
N_TEETH = 32
POINTS_PER_SHAPE = 128 # 256 не лезет в cuda
POINT_DIM = 3
EMBEDDED_POINT_DIM = 2

hidden_dim = POINTS_PER_SHAPE * EMBEDDED_POINT_DIM * N_TEETH // 6

train_loader = DataLoader(align_ds, batch_size=BATCH_SIZE, shuffle=True)
model = autoencoder(POINTS_PER_SHAPE*EMBEDDED_POINT_DIM*N_TEETH, hidden_dim).to(device)


C:\Users\5010858\AppData\Local\Temp/ipykernel_30280/1466985861.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  align_ds = torch.load(align_ds_path)


Using cuda device, ds length - 957


In [3]:
def train_autoencoder(model, train_loader, device, epochs=EPOCHS, lr=1e-3, log_dir="runs/autoencoder"):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=log_dir)

    best_loss = float('inf')  # Initialize best loss to infinity
    best_model_path = None

    for epoch in range(epochs):
        epoch_loss = 0.0
        
        # running_loss_val = 0
        counter = 0
        # counter_val = 0
        
        for batch_idx, (x,y) in enumerate(train_loader):
            counter +=1

            model.train()
            optimizer.zero_grad()
            x = x.view(-1, POINTS_PER_SHAPE*EMBEDDED_POINT_DIM*N_TEETH ).to(device)
            y = y.view(-1, POINTS_PER_SHAPE*EMBEDDED_POINT_DIM*N_TEETH ).to(device)
            pred = model(x)
            loss = loss_fn(pred, y)

             # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # decoder_optimizer.step()
            epoch_loss += loss.item()
        epoch_loss = epoch_loss / counter
        # loss = loss.item()

        if epoch % 10 == 0:
        # loss, current = loss.item(), batch * len(X)
            print(f"Epoch {epoch} - loss {epoch_loss:.5}")

    return None

best_model_path = train_autoencoder(model, train_loader, device)
print(f"Best model saved at: {best_model_path}")


Epoch 0 - loss 1.06e+06
Epoch 10 - loss 57.889
Epoch 20 - loss 57.421
Epoch 30 - loss 57.197
Epoch 40 - loss 59.942
Epoch 50 - loss 57.792
Epoch 60 - loss 63.957
Epoch 70 - loss 57.117
Epoch 80 - loss 67.767
Epoch 90 - loss 57.077
Epoch 100 - loss 56.99
Epoch 110 - loss 57.23
Epoch 120 - loss 57.421
Epoch 130 - loss 56.155
Epoch 140 - loss 56.11
Epoch 150 - loss 56.062
Epoch 160 - loss 56.043
Epoch 170 - loss 56.058
Epoch 180 - loss 55.967
Epoch 190 - loss 56.368
Epoch 200 - loss 55.944
Epoch 210 - loss 55.87
Epoch 220 - loss 55.912
Epoch 230 - loss 55.774
Epoch 240 - loss 55.927
Epoch 250 - loss 55.716
Epoch 260 - loss 55.661
Epoch 270 - loss 55.573
Epoch 280 - loss 55.517
Epoch 290 - loss 55.496
Epoch 300 - loss 55.364
Epoch 310 - loss 55.376
Epoch 320 - loss 55.314
Epoch 330 - loss 55.304
Epoch 340 - loss 55.188
Epoch 350 - loss 55.13
Epoch 360 - loss 55.038
Epoch 370 - loss 55.056
Epoch 380 - loss 55.204
Epoch 390 - loss 54.881
Epoch 400 - loss 55.271
Epoch 410 - loss 54.83
Epoch 4

In [4]:
256*32*32, 15*32

(262144, 480)